# Face Re-Aging Demos with Gradio
To try out the Gradio Demos using Google Colab, run the cells below.
Be sure to select a runtime with GPU (e.g. `Runtime > Change Runtime Type > T4 (GPU)`) to make the model run fast.

You can choose to either run the regular inference demo or the video demo. The demo will open in a new tab.

## Downloading files and model

In [1]:
!git clone https://github.com/timroelofs123/face_reaging

Cloning into 'face_reaging'...


In [2]:
!git clone https://huggingface.co/timroelofs123/face_re-aging

Cloning into 'face_re-aging'...
Updating files: 100% (4/4)
Updating files: 100% (4/4), done.
Filtering content: 100% (2/2)
Filtering content: 100% (2/2), 121.06 MiB | 7.07 MiB/s
Filtering content: 100% (2/2), 121.06 MiB | 5.57 MiB/s, done.


In [3]:
%mv face_re-aging/best_unet_model.pth face_reaging/
%cd face_reaging/

UsageError: Line magic function `%mv` not found.


## Installing requirements

In [4]:
%pip install gradio face_recognition antialiased_cnns kaleido av --quiet

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 3.0.0 requires httpx==0.13.3, but you have httpx 0.27.0 which is incompatible.


## Running demo

Re-age your photo or video with the three demos.

In [13]:
import sys
import os

# Get the absolute path of the current working directory
current_dir = os.path.abspath('.')
# Add the 'face_reaging' directory to the system path
face_reaging_dir = os.path.join(current_dir, 'face_reaging')
if face_reaging_dir not in sys.path:
    sys.path.append(face_reaging_dir)

import gradio as gr
import torch
from model.models import UNet
from scripts.test_functions import process_image, process_video


FileNotFoundError: [Errno 2] No such file or directory: 'assets/mask1024.jpg'

In [18]:
# default settings
window_size = 512
stride = 256
steps = 18
frame_count = 100
model_path = './face_re-aging/best_unet_model.pth'
import os

# Print the current working directory
print(os.getcwd())


c:\Users\shaik\Downloads\Aging\Aging


In [19]:
# The code from this block equivalent to the "run" function in scripts.gradio_demo

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(model_path, map_location=device))
unet_model.eval()

def block_img(image, source_age, target_age):
    return process_image(unet_model, image, video=False, source_age=source_age,
                          target_age=target_age, window_size=window_size, stride=stride)

def block_img_vid(image, source_age):
    return process_image(unet_model, image, video=True, source_age=source_age,
                          target_age=0, window_size=window_size, stride=stride, steps=steps)

def block_vid(video_path, source_age, target_age):
    return process_video(unet_model, video_path, source_age, target_age,
                          window_size=window_size, stride=stride, frame_count=frame_count)

demo_img = gr.Interface(
    fn=block_img,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
        gr.Slider(10, 90, value=80, step=1, label="Target age", info="Choose the age you want to become")
    ],
    outputs="image",
    examples=[
        ['assets/gradio_example_images/1.png', 20, 80],
        ['assets/gradio_example_images/2.png', 75, 40],
        ['assets/gradio_example_images/3.png', 30, 70],
        ['assets/gradio_example_images/4.png', 22, 60],
        ['assets/gradio_example_images/5.png', 28, 75],
        ['assets/gradio_example_images/6.png', 35, 15]
    ],
    description="Input an image of a person and age them from the source age to the target age."
)

demo_img_vid = gr.Interface(
    fn=block_img_vid,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
    ],
    outputs=gr.Video(),
    examples=[
        ['assets/gradio_example_images/1.png', 20],
        ['assets/gradio_example_images/2.png', 75],
        ['assets/gradio_example_images/3.png', 30],
        ['assets/gradio_example_images/4.png', 22],
        ['assets/gradio_example_images/5.png', 28],
        ['assets/gradio_example_images/6.png', 35]
    ],
    description="Input an image of a person and a video will be returned of the person at different ages."
)

demo_vid = gr.Interface(
    fn=block_vid,
    inputs=[
        gr.Video(),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
        gr.Slider(10, 90, value=80, step=1, label="Target age", info="Choose the age you want to become")
    ],
    outputs=gr.Video(),
    examples=[
        ['assets/gradio_example_images/orig.mp4', 35, 60],
    ],
    description="Input a video of a person, and it will be aged frame-by-frame."
)

demo = gr.TabbedInterface([demo_img, demo_img_vid, demo_vid],
                          tab_names=['Image inference demo', 'Image animation demo', 'Video inference demo'],
                          title="Face Re-Aging Demo",
                          )

In [20]:
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\app\python\lib\site-packages\gradio\queueing.py", line 521, in process_events
    response = await route_utils.call_process_api(
  File "c:\app\python\lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\app\python\lib\site-packages\gradio\blocks.py", line 1945, in process_api
    result = await self.call_function(
  File "c:\app\python\lib\site-packages\gradio\blocks.py", line 1513, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\app\python\lib\site-packages\anyio\to_thread.py", line 49, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\app\python\lib\site-packages\anyio\_backends\_asyncio.py", line 2103, in run_sync_in_worker_thread
    return await future
  File "c:\app\python\lib\site-packages\anyio\_backends\_asyncio.py", line 823, in run
    result = context.run(func, *args)
  File "c:\app\py